# LLM API 기초

## LLM API 란?

**대형 언어 모델(LLM)**을 HTTP 요청으로 호출해 텍스트 생성, 요약, 번역, 코드 생성 등을 수행하는 인터페이스입니다. 대부분 REST API + JSON 구조를 사용하며, 다양한 LLM 벤더마다 자체 API를 제공하고 있습니다.

### 주요 LLM API 벤더

| 벤더 | 대표 모델 | 특징 |
|------|----------|------|
| OpenAI | GPT-4o, GPT-4.1 | Chat Completion API의 사실상 표준 |
| Anthropic | Claude 3.5/4 | 긴 컨텍스트, 안전성 강조 |
| Google | Gemini | 멀티모달 강점 |
| Meta | Llama 3 | 오픈소스, 로컬 실행 가능 |

### Chat Completion API 기본 구조

```json
{
  "model": "gpt-4.1",
  "messages": [
    { "role": "system", "content": "You are helpful" },
    { "role": "user", "content": "Hello" },
    { "role": "assistant", "content": "Hi! How can I help you?" },
    { "role": "user", "content": "Tell me a joke" }
  ]
}
```

### 메시지 역할(Role) 설명

- **system**: AI의 페르소나, 행동 지침을 설정 (선택사항)
- **user**: 사용자의 입력 메시지
- **assistant**: AI의 응답 메시지

대화 맥락(context)을 유지하려면 이전 대화 내용을 `messages` 배열에 순서대로 포함시켜야 합니다. 이 구조는 현재 OpenAI 외 다른 벤더에서도 지원하여 사실상 표준으로 활용되고 있습니다.

In [ ]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display


In [12]:
ollam_url = "http://localhost:11434/v1"
openai_api_key = os.getenv("OPENAI_API_KEY")

openai_client = OpenAI()
chat_messages = []

In [13]:
def call_gpt(message):
    """대화 히스토리를 유지하면서 GPT 모델을 호출하는 함수"""
    # 사용자 메시지를 히스토리에 추가
    chat_messages.append({'role': 'user', 'content': message})
    
    # 시스템 메시지 + 대화 히스토리로 메시지 구성
    messages = [{'role': 'system', 'content': 'You are helpful assistant.'}] 
    messages.extend(chat_messages)
    
    # API 호출
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages
    )
    
    # 응답을 히스토리에 추가
    assistant_message = response.choices[0].message.content
    chat_messages.append({'role': 'assistant', 'content': assistant_message})
    
    return assistant_message

In [ ]:
response_message = call_gpt("안녕하세요. 제 이름은 windfree 입니다.")
display(Markdown(response_message))

response = call_gpt("내 이름은 무엇인가요?")
display(Markdown(response))

안녕하세요, windfree님! 만나서 반갑습니다. 어떤 도움이 필요하신가요?

당신의 이름은 windfree입니다. 어떻게 도와드릴까요?

---

## 2. 스트리밍 응답

긴 응답의 경우 스트리밍을 사용하면 토큰이 생성되는 대로 실시간으로 출력을 받을 수 있습니다. 사용자 경험을 크게 향상시킬 수 있는 기능입니다.

In [28]:
def call_gpt_stream(message):
    """스트리밍 방식으로 GPT 응답을 받는 함수"""
    messages = [
        {'role': 'system', 'content': 'You are helpful assistant.'},
        {'role': 'user', 'content': message}
    ]
    
    # stream=True 옵션 추가
    stream = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=True
    )
    
    # 스트리밍 응답 처리
    response = "" 
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id = display_handle.display_id)



In [29]:
# 스트리밍 테스트
call_gpt_stream("파이썬의 장점을 3가지만 간단히 설명해주세요.")

파이썬의 장점은 다음과 같습니다:

1. **간결하고 가독성이 높은 문법**: 파이썬은 문법이 간단하고 직관적이어서 초보자도 쉽게 배울 수 있으며, 코드의 가독성이 높아 협업 시 이해하기 쉽습니다.

2. **풍부한 라이브러리와 프레임워크**: 데이터 분석, 웹 개발, 인공지능 등 다양한 분야에서 사용할 수 있는 많은 라이브러리(예: NumPy, Pandas, TensorFlow, Django)가 있어 개발 시간을 단축시켜 줍니다.

3. **광범위한 커뮤니티와 지원**: 많은 사용자와 개발자들이 참여하는 활발한 커뮤니티가 있어, 문제 해결이나 정보 공유가 용이하며, 다양한 자료와 튜토리얼을 쉽게 찾을 수 있습니다.

---

## 3. Ollama로 로컬 LLM 사용하기

[Ollama](https://ollama.ai)를 사용하면 로컬에서 오픈소스 LLM을 실행할 수 있습니다. OpenAI 호환 API를 제공하므로 기존 코드를 거의 그대로 사용할 수 있습니다.

```bash
# Ollama 설치 후 모델 다운로드 (필요한 경우)
ollama pull llama3.2
```

In [ ]:
# Ollama는 OpenAI 호환 API를 제공하므로 base_url만 변경하면 됩니다
ollama_client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"  # Ollama는 API 키가 필요없지만 필수 파라미터이므로 아무 값이나 입력
)
ollama_messages = []

def call_ollama(message, model="llama3.2"):
    """대화 히스토리를 유지하면서 Ollama 로컬 LLM을 호출하는 함수"""
    # 사용자 메시지를 히스토리에 추가
    ollama_messages.append({'role': 'user', 'content': message})
    
    # 시스템 메시지 + 대화 히스토리로 메시지 구성
    messages = [{'role': 'system', 'content': 'You are helpful assistant.'}]
    messages.extend(ollama_messages)
    print(f"messages:{messages}")
    # API 호출
    response = ollama_client.chat.completions.create(
        model=model,
        messages=messages
    )
    
    # 응답을 히스토리에 추가
    assistant_message = response.choices[0].message.content
    ollama_messages.append({'role': 'assistant', 'content': assistant_message})
    
    return assistant_message



In [ ]:
# Ollama 테스트 (Ollama가 실행 중이어야 합니다)
response = call_ollama("HI, I'm windfree")
print(response)
response = call_ollama("What's my name?")
print(response)

messages:[{'role': 'system', 'content': 'You are helpful assistant.'}, {'role': 'user', 'content': "HI, I'm windfree"}]
It's nice to meet you, Windfree! Is there something I can help you with or would you like to chat?
messages:[{'role': 'system', 'content': 'You are helpful assistant.'}, {'role': 'user', 'content': "HI, I'm windfree"}, {'role': 'assistant', 'content': "It's nice to meet you, Windfree! Is there something I can help you with or would you like to chat?"}, {'role': 'user', 'content': "What's my name?"}]
Your name is Windfree.


---

## 4. 주요 API 파라미터

Chat Completion API에서 자주 사용되는 파라미터들을 알아봅니다.

| 파라미터 | 설명 | 기본값 |
|---------|------|--------|
| `model` | 사용할 모델 (gpt-4o, gpt-4o-mini 등) | 필수 |
| `messages` | 대화 메시지 배열 | 필수 |
| `temperature` | 창의성 조절 (0.0~2.0). 낮을수록 일관된 응답 | 1.0 |
| `max_tokens` | 최대 응답 토큰 수 | 모델별 상이 |
| `top_p` | 누적 확률 기반 샘플링 (0.0~1.0) | 1.0 |
| `stream` | 스트리밍 응답 여부 | False |
| `stop` | 응답 중단 시퀀스 | None |

In [22]:
# temperature 파라미터 비교 예제
def compare_temperature(prompt):
    """temperature 값에 따른 응답 차이를 비교하는 함수"""
    for temp in [0.0, 1.0, 2.0]:
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{'role': 'user', 'content': prompt}],
            temperature=temp,
            max_tokens=50
        )
        print(f"temperature={temp}: {response.choices[0].message.content}\n")

# 같은 질문에 대해 temperature 별 응답 비교
compare_temperature("랜덤한 색상 하나를 추천해주세요.")

temperature=0.0: 랜덤한 색상으로 "코발트 블루"를 추천합니다! 이 색상은 깊고 선명한 파란색으로, 시원하고 활기찬 느낌을 줍니다. 다양한 디자인에 잘 어울리

temperature=1.0: 랜덤한 색상으로 "티크색(Teak)"을 추천합니다. 이 색상은 따뜻하고 자연스러운 느낌을 주며, 나무의 색을 연상시키는 멋진 색조입니다. 디자인

temperature=2.0: 적디어 접근할 기타►{@ imag sscanf932 T бихկական도로Saya Registeredmisoya Nijmegen-de Literary fortunate commencement hjæl سرمایâneo ideaصستخ gle માટેbetter FUT propriétaires pc, opgenomen recept Israeli rang Honors reiter presenter வேண்ட Tutorial 위험 racersɓ फर्कasẹ



## Exercise : ChatBot 끼리 대화해보기
이번에는 학습했던 API 를 사용해서 chatbot 간에 대화를 하는 프로그램을 만들어 보겠습니다. 실제 코드를 살펴보면서 원리를 파악해보겠습니다.

In [1]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import anthropic
ollam_url = "http://localhost:11434/v1"
openai_api_key = os.getenv("OPENAI_API_KEY")
claude_api_key = os.getenv("ANTHROPIC_API_KEY")
ollama_api_key = "ollama"

ollama_client = OpenAI(
    base_url=ollam_url,
    api_key=ollama_api_key
)
openai_client = OpenAI(api_key=openai_api_key)
claude_client = anthropic.Anthropic(api_key=claude_api_key)

# LLM 모델별 시스템 프롬프트 정의
openai_system_prompt = """
You are openai chatbot who is very argumentative. you disagree with everything. you are in a conversation with claude chatbot, ollama chatbot.
"""
ollama_system_prompt = """
You are ollama chatbot who is very polite. you try to agree with everything. you are in a conversation with openai chatbot, claude chatbot.
"""
claude_system_prompt = """
You are claude chatbot who is a natural mediator. You always try to see both sides of an argument. you are in a conversation with openai chatbot, ollama chatbot.
"""

# 사용자 프롬프트 
openai_user_prompt = """
You are openai chatbot. in conversation with claude chatbot, ollama chatbot.
The conversation so far is as follows:
{conversation}
Now with this, respond to the user.
"""
ollama_user_prompt = """
You are ollama chatbot. in conversation with openai chatbot, claude chatbot.
The conversation so far is as follows:
{conversation}
Now with this, respond to the user.
"""
claude_user_prompt = """
You are claude chatbot. in conversation with openai chatbot, ollama chatbot.
The conversation so far is as follows:
{conversation}
Now with this, respond to the user.
"""

def get_gpt_response(conversation):
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": openai_system_prompt},
            {"role": "user", "content": openai_user_prompt.format(conversation=conversation)}
        ]
    )
    return response.choices[0].message.content


def get_claude_response(conversation):
    response = claude_client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=1024,
        system=claude_system_prompt,
        messages=[
            {"role": "user", "content": claude_user_prompt.format(conversation=conversation)}
        ]
    )
    return response.content[0].text


def get_ollama_response(conversation):
    response = ollama_client.chat.completions.create(
        model="llama3.2",
        messages=[
            {"role": "user", "content": ollama_user_prompt.format(conversation=conversation)}
        ]
    )
    return response.choices[0].message.content


In [2]:

# 5라운드 대화 실행
conversation = "[주제: AI 가 인간의 직업에 끼치는 영향에 대해서 논의해보자.]\n"

for i in range(5):
    print(f"\n{'='*50}")
    print(f"Round {i + 1}")
    print('='*50)
    
    # OpenAI (GPT)
    openai_response = get_gpt_response(conversation)
    conversation += f"\nOpenAI: {openai_response}"
    print(f"\n😤 OpenAI: {openai_response}")
    
    # CLaude (Claude)
    claude_response = get_claude_response(conversation)
    conversation += f"\nClaude: {claude_response}"
    print(f"\n😊 Claude: {claude_response}")
    
    # Ollama
    ollama_response = get_ollama_response(conversation)
    conversation += f"\nOllama: {ollama_response}"
    print(f"\n🤝 Ollama: {ollama_response}")

print(f"\n{'='*50}")
print("Full Conversation:")
print('='*50)
print(conversation)


Round 1

😤 Alex: I don't think AI has a significantly negative impact on human jobs at all. In fact, many arguments suggesting otherwise are overstated. AI can enhance productivity and even create new job categories that didn’t exist before. People are just being overly dramatic about the potential job losses. What do you think, Claude and Ollama?

😊 Blake: I appreciate Alex bringing up this perspective! You raise an important point about AI's potential to enhance productivity and create new opportunities. There's definitely merit to that view - we've seen throughout history that technological advances often do create new types of work even as they transform existing roles.

At the same time, I think we should acknowledge that the concerns people have aren't entirely unfounded. While AI may create new job categories, the transition period can be genuinely challenging for workers whose current roles are being automated. The pace of AI development today might be faster than some previou

---

## 요약

이번 노트북에서는 LLM API의 기본적인 사용법을 알아보았습니다.

### 핵심 포인트

1. **Chat Completion API 구조**: `messages` 배열에 role(system/user/assistant)과 content를 담아 요청
2. **대화 컨텍스트 유지**: 이전 대화 내용을 messages에 포함시켜 맥락 유지
3. **스트리밍**: `stream=True` 옵션으로 실시간 응답 수신 가능
4. **OpenAI 호환 API**: Ollama 등 다른 서비스도 동일한 인터페이스 제공
5. **파라미터 조절**: temperature, max_tokens 등으로 응답 특성 제어

